In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [2]:
files = glob('../pure-text/dumping*.txt')
files = [f for f in files if 'twitter' not in f and 'common' not in f]
files

['../pure-text/dumping-instagram.txt',
 '../pure-text/dumping-parliament.txt',
 '../pure-text/dumping-iium.txt',
 '../pure-text/dumping-wiki.txt',
 '../pure-text/dumping-news.txt',
 '../pure-text/dumping-watpadd.txt',
 '../pure-text/dumping-pdf.txt']

In [3]:
os.path.split(files[0])[1]

'dumping-instagram.txt'

In [4]:
import re

def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [6]:
for file in files:
    with open(file) as fopen:
        data = fopen.read().split('\n')
    results, result = [], []
    for i in data:
        if not len(i) and len(result):
            results.append('. '.join(result))
            result = []
        else:
            result.append(i)
    print(file, len(results))
    s = os.path.split(file)[1]
    filename = f'{s}-pair.tsv'
    
    with tf.io.gfile.GFile(filename, 'w') as outfile:
        for i in range(len(results)):
            outfile.write('%s\t\n' % (cleaning(results[i])))

../pure-text/dumping-instagram.txt 695600
../pure-text/dumping-parliament.txt 70046
../pure-text/dumping-iium.txt 17837
../pure-text/dumping-wiki.txt 326735
../pure-text/dumping-news.txt 105469
../pure-text/dumping-watpadd.txt 55091
../pure-text/dumping-pdf.txt 55538


In [7]:
glob('dumping*pair.tsv')

['dumping-news.txt-pair.tsv',
 'dumping-iium.txt-pair.tsv',
 'dumping-parliament.txt-pair.tsv',
 'dumping-wiki.txt-pair.tsv',
 'dumping-watpadd.txt-pair.tsv',
 'dumping-pdf.txt-pair.tsv',
 'dumping-instagram.txt-pair.tsv']

In [8]:
def pair_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        glob('dumping*pair.tsv')
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['text'], ex)))
    return ds


t5.data.TaskRegistry.remove('pair_dataset')
t5.data.TaskRegistry.add(
    'pair_dataset',
    dataset_fn = pair_dataset,
    splits = ['train'],
    text_preprocessor = [prep.next_sentence_prediction],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)

/home/husein/.local/lib/python3.6/site-packages/t5/models/mesh_transformer.py:210: UserWarning: get_sentencepiece_model_path is deprecated. Please pass the mixture or task vocabulary directly to the Mesh TensorFlow Transformer instead.
  "get_sentencepiece_model_path is deprecated. Please pass the mixture or "


In [ ]:
from tqdm import tqdm

nq_task = t5.data.TaskRegistry.get("pair_dataset")
ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 768, "targets": 768})

batch_size, index, part = 100000, 0, 0
fopen = open(f'pair-{part}.parse', 'w')
for ex in tqdm(tfds.as_numpy(ds)):
    i = sp.DecodeIds(ex['inputs'].tolist())
    t = sp.DecodeIds(ex['targets'].tolist())
    text = f'{i} [[EENNDD]] {t}\n'
    fopen.write(text)
    
    if index == batch_size:
        fopen.close()
        part += 1
        index = 0
        fopen = open(f'pair-{part}.parse', 'w')
    
    index += 1
    
fopen.close()

      lambda x: x[text_key], num_parallel_calls=tf.data.experimental.AUTOTUNE)

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.


      lambda x: x[text_key], num_parallel_calls=tf.data.experimental.AUTOTUNE)

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.


      lambda x: x[text_key], num_parallel_calls=tf.data.experimental.AUTOTUNE)

If this is a lambda function, the error may be avoided by creating the lambda in a standalone statement.


11196416it [1:50:58, 1815.82it/s]